# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [21]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages', engine)
X = df['message']
Y = df.drop(['id','message','original','genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    '''extracts words from english message given and provides clean tokens'''
    
    # replace non letter or non number characters 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # get tokens from text and remove stopwords
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # clean and lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    
    return(clean_tokens)
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier())),    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)

In [7]:
y_pred = pd.DataFrame(y_pred, columns = y_test.columns)
for col in y_test.columns:
    print("Feature: "+ col)
    print(classification_report(y_test[col],y_pred[col]))

Feature: related
             precision    recall  f1-score   support

          0       0.64      0.47      0.55      1266
          1       0.84      0.91      0.87      3938
          2       0.21      0.33      0.25        40

avg / total       0.79      0.80      0.79      5244

Feature: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4349
          1       0.80      0.43      0.56       895

avg / total       0.88      0.88      0.87      5244

Feature: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      0.99      0.99      5244

Feature: aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3113
          1       0.74      0.59      0.66      2131

avg / total       0.75      0.75      0.74      5244

Feature: medical_

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7b2ef97a1d90>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [9]:
parameters = {
    'clf__estimator__criterion' : ['gini', 'entropy'],
    'clf__estimator__n_estimators' : [10,50]
}

cv = GridSearchCV(pipeline, param_grid = parameters)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__n_estimators': [10, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [11]:
cv.best_params_

{'clf__estimator__criterion': 'gini', 'clf__estimator__n_estimators': 50}

In [12]:
y_pred_tuned = cv.predict(X_test)

In [13]:
y_pred_tuned = pd.DataFrame(y_pred_tuned, columns = y_test.columns)
for col in y_test.columns:
    print("Feature: "+ col)
    print(classification_report(y_test[col],y_pred_tuned[col]))

Feature: related
             precision    recall  f1-score   support

          0       0.71      0.43      0.54      1266
          1       0.84      0.94      0.88      3938
          2       0.41      0.45      0.43        40

avg / total       0.80      0.81      0.80      5244

Feature: request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      4349
          1       0.82      0.48      0.61       895

avg / total       0.89      0.89      0.88      5244

Feature: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

Feature: aid_related
             precision    recall  f1-score   support

          0       0.79      0.84      0.82      3113
          1       0.75      0.68      0.71      2131

avg / total       0.77      0.78      0.77      5244

Feature: medical_

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
# check AdaBoostClassifier for better performance
pipeline_II = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier())),    
])

In [18]:
pipeline_II.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [19]:
y_pred_II = pipeline_II.predict(X_test)

In [20]:
y_pred_II = pd.DataFrame(y_pred, columns = y_test.columns)
for col in y_test.columns:
    print("Feature: "+ col)
    print(classification_report(y_test[col],y_pred_II[col]))

Feature: related
             precision    recall  f1-score   support

          0       0.64      0.47      0.55      1266
          1       0.84      0.91      0.87      3938
          2       0.21      0.33      0.25        40

avg / total       0.79      0.80      0.79      5244

Feature: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4349
          1       0.80      0.43      0.56       895

avg / total       0.88      0.88      0.87      5244

Feature: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      0.99      0.99      5244

Feature: aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3113
          1       0.74      0.59      0.66      2131

avg / total       0.75      0.75      0.74      5244

Feature: medical_

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [24]:
with open("Disaster_response_model.pkl", 'wb') as exp_file:
    pickle.dump(pipeline, exp_file)


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.